Homeowners who are looking to sell their homes have a chance to take in more money than they paid for it. For many families, it's one of the few opportunities they have to make several, if not tens of thousands of dollars in one move. When so much is at stake, they turn to professionals (i.e. real estate agents) for advice on how to get a fair price and make the most of this opprtunity. 

This notebook will determine what the most important factors are that dive home prices. With this information, a real estate agent in King County, Oregon can tell their client what they can do to increase the value of their home before they put it on the market.

To start, we'll import the necessary libraries to do our analysis, and then take a preliminary look at the data.

In [1]:
import pandas as pd
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.special import logsumexp
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
from scipy import stats as stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
%matplotlib inline

In [2]:
df = pd.read_csv('data/kc_house_data.csv')

df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [3]:
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.159700e+04,2.159700e+04,21597.000000,21597.000000,21597.000000,2.159700e+04,21597.000000,21597.000000,21597.000000,17755.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000
mean,4.580474e+09,5.402966e+05,3.373200,2.115826,2080.321850,1.509941e+04,1.494096,1788.596842,1970.999676,83.636778,98077.951845,47.560093,-122.213982,1986.620318,12758.283512
std,2.876736e+09,3.673681e+05,0.926299,0.768984,918.106125,4.141264e+04,0.539683,827.759761,29.375234,399.946414,53.513072,0.138552,0.140724,685.230472,27274.441950
min,1.000102e+06,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,370.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,1190.000000,1951.000000,0.000000,98033.000000,47.471100,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,1560.000000,1975.000000,0.000000,98065.000000,47.571800,-122.231000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068500e+04,2.000000,2210.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,9410.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

It looks like we'll have to convert some columns into floats so we can build models with them. We'll also have to replace null values for the waterfront, view, and yr_renovated columns.

I also noted a max value of 33 bedrooms. I'm going to check that it's a misinput of 3, or a complete outlier.

Going forward, I am going to assume that if a home doesn't have a waterfront value, it means it is not waterfront. It would logically make sense that a homeowner with a waterfront property would make sure to fill out that section, but a homeowner not near the water would just skip over it or leave it blank. This also applies for things like basement size and year renovated.

In [5]:
df['date'].value_counts()

6/23/2014     142
6/26/2014     131
6/25/2014     131
7/8/2014      127
4/27/2015     126
             ... 
1/10/2015       1
5/27/2015       1
5/24/2015       1
1/17/2015       1
11/30/2014      1
Name: date, Length: 372, dtype: int64

In [6]:
df['waterfront'].value_counts()

NO     19075
YES      146
Name: waterfront, dtype: int64

In [7]:
df['waterfront'] = df['waterfront'].fillna('NO')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     21597 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [9]:
df['view'].value_counts()

NONE         19422
AVERAGE        957
GOOD           508
FAIR           330
EXCELLENT      317
Name: view, dtype: int64

In [10]:
df['view'] = df['view'].fillna('NONE')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     21597 non-null  object 
 9   view           21597 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [12]:
df['yr_renovated'].value_counts()

0.0       17011
2014.0       73
2003.0       31
2013.0       31
2007.0       30
          ...  
1946.0        1
1959.0        1
1971.0        1
1951.0        1
1954.0        1
Name: yr_renovated, Length: 70, dtype: int64

In [13]:
df['yr_renovated'] = df['yr_renovated'].fillna(0.0)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     21597 non-null  object 
 9   view           21597 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   21597 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [15]:
df.loc[df['bedrooms'] == 33]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
15856,2402100895,6/25/2014,640000.0,33,1.75,1620,6000,1.0,NO,NONE,...,7 Average,1040,580.0,1947,0.0,98103,47.6878,-122.331,1330,4700


In [16]:
#I can believe that's an input error, and the home is actually 3 bedrooms instead of 33

df['bedrooms'] = df['bedrooms'].replace(33, 3)

In [17]:
df['bedrooms'].describe()

count    21597.000000
mean         3.371811
std          0.904096
min          1.000000
25%          3.000000
50%          3.000000
75%          4.000000
max         11.000000
Name: bedrooms, dtype: float64

In [18]:
#change waterfront, condition, view, grade to be numerical values

df['waterfront'].value_counts()

NO     21451
YES      146
Name: waterfront, dtype: int64

In [19]:
df['waterfront'] = df['waterfront'].replace(['NO','YES'],['0','1']).astype(float)

In [20]:
df['condition'].value_counts()

Average      14020
Good          5677
Very Good     1701
Fair           170
Poor            29
Name: condition, dtype: int64

In [21]:
df['condition'] = df['condition'].replace(['Poor','Fair','Average','Good','Very Good'],['1','2','3','4','5']).astype(float)

In [22]:
df['view'].value_counts()

NONE         19485
AVERAGE        957
GOOD           508
FAIR           330
EXCELLENT      317
Name: view, dtype: int64

In [23]:
#assuming that the scale goes from poor, fair, average, good, excellent

df['view'] = df['view'].replace(['NONE','FAIR','AVERAGE','GOOD','EXCELLENT'],['0','2','3','4','5']).astype(float)

In [24]:
df['grade'].value_counts()

7 Average        8974
8 Good           6065
9 Better         2615
6 Low Average    2038
10 Very Good     1134
11 Excellent      399
5 Fair            242
12 Luxury          89
4 Low              27
13 Mansion         13
3 Poor              1
Name: grade, dtype: int64

In [25]:
#it comes with its own numerical value, so we just have to isolate the number

grading = df['grade'].str.split(' ', n=2, expand=True)

grading

,0,1,2
0,7,Average,None
1,7,Average,None
2,6,Low,Average
3,7,Average,None
4,8,Good,None
...,...,...,...
21592,8,Good,None
21593,8,Good,None
21594,7,Average,None
21595,8,Good,None


In [26]:
grading[0].value_counts()

7     8974
8     6065
9     2615
6     2038
10    1134
11     399
5      242
12      89
4       27
13      13
3        1
Name: 0, dtype: int64

In [27]:
df['grade'] = grading[0].astype(float)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     21597 non-null  float64
 9   view           21597 non-null  float64
 10  condition      21597 non-null  float64
 11  grade          21597 non-null  float64
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   21597 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [29]:
df['sqft_basement'].value_counts()

0.0       12826
?           454
600.0       217
500.0       209
700.0       208
          ...  
518.0         1
1248.0        1
283.0         1
248.0         1
2850.0        1
Name: sqft_basement, Length: 304, dtype: int64

In [30]:
#assume that missing values means they don't have a basement, so they left that field blank

df['sqft_basement'] = df['sqft_basement'].replace('?','0.0').astype(float)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     21597 non-null  float64
 9   view           21597 non-null  float64
 10  condition      21597 non-null  float64
 11  grade          21597 non-null  float64
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  float64
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   21597 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

Now that our data is clean, we can start building models to determine what drives price. 

ID, data, and zipcode will not be super useful for our purposes. Zipcode in particular will not be as reliable of a location factor as latitude and longitude.

Price will not be part of our features since it is the target.

In [32]:
base_features = sm.add_constant(df.drop(['id', 'date','price','zipcode'], axis=1))
target = df['price']

base_model = sm.OLS(target, base_features).fit()

In [33]:
base_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.696
Model:                            OLS   Adj. R-squared:                  0.696
Method:                 Least Squares   F-statistic:                     2905.
Date:                Sun, 10 Jul 2022   Prob (F-statistic):               0.00
Time:                        16:47:09   Log-Likelihood:            -2.9453e+05
No. Observations:               21597   AIC:                         5.891e+05
Df Residuals:                   21579   BIC:                         5.892e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -3.678e+07    1.6e+06    -23.036      0.000   -3.99e+07   -3.36e+07
bedrooms      -3.753e+04   1991.619    -18.845      0.000   -4.14e+04   -3.36e+04
bathrooms      4.374e+04   3289.667     13.295      0.000    3.73e+04    5.02e+04
sqft_living      99.8317     18.204      5.484      0.000      64.151     135.513
sqft_lot          0.1251      0.048      2.593      0.010       0.031       0.220
floors         1094.3633   3609.605      0.303      0.762   -5980.730    8169.456
waterfront     6.402e+05    1.8e+04     35.530      0.000    6.05e+05    6.76e+05
view           3.527e+04   1539.053     22.915      0.000    3.23e+04    3.83e+04
condition      3.145e+04   2347.830     13.397      0.000    2.69e+04    3.61e+04
grade          9.789e+04   2175.616     44.994      0.000    9.36e+04    1.02e+05
sqft_above       81.2341     18.192      4.465      0.000      45.576     116.892
sqft_basement    49.3018     18.030      2.734      0.006      13.962      84.641
yr_built      -2484.9807     71.885    -34.569      0.000   -2625.882   -2344.080
yr_renovated     24.8096      4.004      6.196      0.000      16.961      32.658
lat            5.575e+05   1.05e+04     52.843      0.000    5.37e+05    5.78e+05
long          -1.182e+05    1.2e+04     -9.836      0.000   -1.42e+05   -9.46e+04
sqft_living15    27.1451      3.461      7.842      0.000      20.361      33.930
sqft_lot15       -0.4032      0.074     -5.459      0.000      -0.548      -0.258
==============================================================================
Omnibus:                    18424.550   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1825810.403
Skew:                           3.593   Prob(JB):                         0.00
Kurtosis:                      47.467   Cond. No.                     5.86e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.86e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

This is promising so far. Our model explains almost 70% of the variance. However, number of floors ended up being an unreliable feature (p > 0.05). So we will remove that feature and see what happens next.

In [34]:
model2_x = base_features.drop('floors', axis=1)

In [35]:
model2_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   const          21597 non-null  float64
 1   bedrooms       21597 non-null  int64  
 2   bathrooms      21597 non-null  float64
 3   sqft_living    21597 non-null  int64  
 4   sqft_lot       21597 non-null  int64  
 5   waterfront     21597 non-null  float64
 6   view           21597 non-null  float64
 7   condition      21597 non-null  float64
 8   grade          21597 non-null  float64
 9   sqft_above     21597 non-null  int64  
 10  sqft_basement  21597 non-null  float64
 11  yr_built       21597 non-null  int64  
 12  yr_renovated   21597 non-null  float64
 13  lat            21597 non-null  float64
 14  long           21597 non-null  float64
 15  sqft_living15  21597 non-null  int64  
 16  sqft_lot15     21597 non-null  int64  
dtypes: float64(10), int64(7)
memory usage: 2.8 MB


In [36]:
model2 = sm.OLS(target, model2_x).fit()

In [37]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.696
Model:                            OLS   Adj. R-squared:                  0.696
Method:                 Least Squares   F-statistic:                     3087.
Date:                Sun, 10 Jul 2022   Prob (F-statistic):               0.00
Time:                        16:47:09   Log-Likelihood:            -2.9453e+05
No. Observations:               21597   AIC:                         5.891e+05
Df Residuals:                   21580   BIC:                         5.892e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -3.686e+07   1.57e+06    -23.456      0.000   -3.99e+07   -3.38e+07
bedrooms      -3.755e+04   1990.758    -18.862      0.000   -4.15e+04   -3.36e+04
bathrooms        4.4e+04   3172.672     13.868      0.000    3.78e+04    5.02e+04
sqft_living      99.4216     18.153      5.477      0.000      63.840     135.003
sqft_lot          0.1248      0.048      2.587      0.010       0.030       0.219
waterfront     6.402e+05    1.8e+04     35.531      0.000    6.05e+05    6.76e+05
view           3.528e+04   1538.229     22.937      0.000    3.23e+04    3.83e+04
condition       3.14e+04   2342.017     13.409      0.000    2.68e+04     3.6e+04
grade          9.795e+04   2167.291     45.193      0.000    9.37e+04    1.02e+05
sqft_above       81.8241     18.087      4.524      0.000      46.372     117.276
sqft_basement    49.3027     18.029      2.735      0.006      13.964      84.641
yr_built      -2480.5287     70.368    -35.251      0.000   -2618.456   -2342.602
yr_renovated     24.8607      4.000      6.214      0.000      17.020      32.702
lat            5.578e+05   1.05e+04     53.171      0.000    5.37e+05    5.78e+05
long          -1.187e+05   1.19e+04     -9.973      0.000   -1.42e+05   -9.54e+04
sqft_living15    27.0173      3.435      7.864      0.000      20.283      33.751
sqft_lot15       -0.4040      0.074     -5.472      0.000      -0.549      -0.259
==============================================================================
Omnibus:                    18415.726   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1822599.236
Skew:                           3.591   Prob(JB):                         0.00
Kurtosis:                      47.428   Cond. No.                     5.76e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.76e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Not much changed. Let's try engineering a new feature called "livable area." It will be the product of bedrooms, bathrooms, and sqft_living, since bedrooms and bathrooms are part of what makes up living square footage.

In [38]:
model2_x['livable_area'] = model2_x['bedrooms'] * model2_x['bathrooms'] * model2_x['sqft_living']

In [39]:
model2_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   const          21597 non-null  float64
 1   bedrooms       21597 non-null  int64  
 2   bathrooms      21597 non-null  float64
 3   sqft_living    21597 non-null  int64  
 4   sqft_lot       21597 non-null  int64  
 5   waterfront     21597 non-null  float64
 6   view           21597 non-null  float64
 7   condition      21597 non-null  float64
 8   grade          21597 non-null  float64
 9   sqft_above     21597 non-null  int64  
 10  sqft_basement  21597 non-null  float64
 11  yr_built       21597 non-null  int64  
 12  yr_renovated   21597 non-null  float64
 13  lat            21597 non-null  float64
 14  long           21597 non-null  float64
 15  sqft_living15  21597 non-null  int64  
 16  sqft_lot15     21597 non-null  int64  
 17  livable_area   21597 non-null  float64
dtypes: flo

In [40]:
model3_x = model2_x.drop(['bedrooms','bathrooms','sqft_living'], axis=1)

In [41]:
model3 = sm.OLS(target, model3_x).fit()

model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.706
Model:                            OLS   Adj. R-squared:                  0.706
Method:                 Least Squares   F-statistic:                     3703.
Date:                Sun, 10 Jul 2022   Prob (F-statistic):               0.00
Time:                        16:47:09   Log-Likelihood:            -2.9417e+05
No. Observations:               21597   AIC:                         5.884e+05
Df Residuals:                   21582   BIC:                         5.885e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -3.692e+07   1.54e+06    -23.986      0.000   -3.99e+07   -3.39e+07
sqft_lot          0.1578      0.047      3.328      0.001       0.065       0.251
waterfront     6.599e+05   1.77e+04     37.295      0.000    6.25e+05    6.95e+05
view           3.727e+04   1506.107     24.744      0.000    3.43e+04    4.02e+04
condition      3.335e+04   2295.315     14.529      0.000    2.88e+04    3.78e+04
grade          1.113e+05   2105.446     52.845      0.000    1.07e+05    1.15e+05
sqft_above       71.5157      4.203     17.016      0.000      63.278      79.754
sqft_basement    43.9037      4.494      9.768      0.000      35.094      52.713
yr_built      -2181.8217     63.236    -34.503      0.000   -2305.770   -2057.874
yr_renovated     31.2010      3.896      8.008      0.000      23.564      38.838
lat             5.68e+05   1.03e+04     55.186      0.000    5.48e+05    5.88e+05
long          -1.101e+05   1.17e+04     -9.414      0.000   -1.33e+05   -8.72e+04
sqft_living15    39.6957      3.393     11.698      0.000      33.045      46.347
sqft_lot15       -0.3175      0.072     -4.384      0.000      -0.459      -0.176
livable_area      4.6282      0.131     35.359      0.000       4.372       4.885
==============================================================================
Omnibus:                    14601.712   Durbin-Watson:                   1.989
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           912658.916
Skew:                           2.580   Prob(JB):                         0.00
Kurtosis:                      34.426   Cond. No.                     5.88e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.88e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

It helps the model explain only slightly more of the variance, but it is a rather robust feature with a very small standard error. 

An r-squared 0.706 gives me enough confidence to make recommendations.

The next thing to tackle is the possible colinearity. We will determine the VIF, or variance inflation factor, for each feature. This will tell us the amount of multicolinearity of each individual variable. Generally, a score below 5 is safe, 5-10 is a "use with caution" zone, and anything above 10 must be removed. 

In [42]:
vif_data = pd.DataFrame()

In [43]:
vif_data['feature'] = model3_x.columns

In [44]:
vif_data['VIF'] = [variance_inflation_factor(model3_x.values, i)
                          for i in range(len(model3_x.columns))]

In [45]:
vif_data

,feature,VIF
0,const,1.288807e+06
1,sqft_lot,2.097904e+00
2,waterfront,1.143830e+00
3,view,1.349760e+00
4,condition,1.213039e+00
5,grade,3.319453e+00
6,sqft_above,6.584822e+00
7,sqft_basement,2.125884e+00
8,yr_built,1.877289e+00
9,yr_renovated,1.094486e+00


The only feature with a VIF above 5 is sqft_above. It's not a make-or-break for our model, but we'll go ahead and remove it anyway just to be safe.

In [46]:
model4_x = model3_x.drop('sqft_above', axis=1)

model4_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   const          21597 non-null  float64
 1   sqft_lot       21597 non-null  int64  
 2   waterfront     21597 non-null  float64
 3   view           21597 non-null  float64
 4   condition      21597 non-null  float64
 5   grade          21597 non-null  float64
 6   sqft_basement  21597 non-null  float64
 7   yr_built       21597 non-null  int64  
 8   yr_renovated   21597 non-null  float64
 9   lat            21597 non-null  float64
 10  long           21597 non-null  float64
 11  sqft_living15  21597 non-null  int64  
 12  sqft_lot15     21597 non-null  int64  
 13  livable_area   21597 non-null  float64
dtypes: float64(10), int64(4)
memory usage: 2.3 MB


In [47]:
model4 = sm.OLS(target, model4_x).fit()

model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.702
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     3913.
Date:                Sun, 10 Jul 2022   Prob (F-statistic):               0.00
Time:                        16:47:09   Log-Likelihood:            -2.9431e+05
No. Observations:               21597   AIC:                         5.887e+05
Df Residuals:                   21583   BIC:                         5.888e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -3.412e+07   1.54e+06    -22.147      0.000   -3.71e+07   -3.11e+07
sqft_lot          0.1960      0.048      4.110      0.000       0.103       0.289
waterfront     6.699e+05   1.78e+04     37.630      0.000    6.35e+05    7.05e+05
view            3.63e+04   1515.050     23.956      0.000    3.33e+04    3.93e+04
condition      3.394e+04   2310.340     14.691      0.000    2.94e+04    3.85e+04
grade          1.256e+05   1942.588     64.651      0.000    1.22e+05    1.29e+05
sqft_basement    -2.2493      3.608     -0.623      0.533      -9.321       4.822
yr_built      -2180.7830     63.658    -34.258      0.000   -2305.557   -2056.009
yr_renovated     33.5821      3.920      8.568      0.000      25.899      41.265
lat            5.581e+05   1.03e+04     53.952      0.000    5.38e+05    5.78e+05
long          -9.065e+04   1.17e+04     -7.737      0.000   -1.14e+05   -6.77e+04
sqft_living15    62.0167      3.150     19.686      0.000      55.842      68.192
sqft_lot15       -0.2845      0.073     -3.904      0.000      -0.427      -0.142
livable_area      6.1791      0.095     65.338      0.000       5.994       6.364
==============================================================================
Omnibus:                    13952.842   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1002064.255
Skew:                           2.357   Prob(JB):                         0.00
Kurtosis:                      36.035   Cond. No.                     5.85e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.85e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Our model still explains about the same amount of variance, so that's safe to use. However, sqft_basement is now unreliable as a variable, so we'll have to remove that.

In [53]:
model5_x = model4_x.drop('sqft_basement', axis=1)

model5_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   const          21597 non-null  float64
 1   sqft_lot       21597 non-null  int64  
 2   waterfront     21597 non-null  float64
 3   view           21597 non-null  float64
 4   condition      21597 non-null  float64
 5   grade          21597 non-null  float64
 6   yr_built       21597 non-null  int64  
 7   yr_renovated   21597 non-null  float64
 8   lat            21597 non-null  float64
 9   long           21597 non-null  float64
 10  sqft_living15  21597 non-null  int64  
 11  sqft_lot15     21597 non-null  int64  
 12  livable_area   21597 non-null  float64
dtypes: float64(9), int64(4)
memory usage: 2.1 MB


In [54]:
model5 = sm.OLS(target, model5_x).fit()

model5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.702
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     4239.
Date:                Sun, 10 Jul 2022   Prob (F-statistic):               0.00
Time:                        16:49:47   Log-Likelihood:            -2.9431e+05
No. Observations:               21597   AIC:                         5.887e+05
Df Residuals:                   21584   BIC:                         5.888e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -3.398e+07   1.52e+06    -22.283      0.000    -3.7e+07    -3.1e+07
sqft_lot          0.1962      0.048      4.114      0.000       0.103       0.290
waterfront     6.701e+05   1.78e+04     37.649      0.000    6.35e+05    7.05e+05
view           3.614e+04   1494.386     24.184      0.000    3.32e+04    3.91e+04
condition      3.374e+04   2287.665     14.749      0.000    2.93e+04    3.82e+04
grade          1.257e+05   1938.341     64.834      0.000    1.22e+05    1.29e+05
yr_built      -2177.5578     63.446    -34.321      0.000   -2301.917   -2053.199
yr_renovated     33.5594      3.919      8.562      0.000      25.877      41.242
lat            5.576e+05   1.03e+04     54.050      0.000    5.37e+05    5.78e+05
long          -8.967e+04   1.16e+04     -7.723      0.000   -1.12e+05   -6.69e+04
sqft_living15    61.9312      3.147     19.677      0.000      55.762      68.100
sqft_lot15       -0.2847      0.073     -3.906      0.000      -0.428      -0.142
livable_area      6.1589      0.089     69.304      0.000       5.985       6.333
==============================================================================
Omnibus:                    13964.327   Durbin-Watson:                   1.987
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1001147.954
Skew:                           2.361   Prob(JB):                         0.00
Kurtosis:                      36.019   Cond. No.                     5.79e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.79e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Now we will scale the features to determine how impactful they are to the model. The larger the value, the more impactful.

These will effectively be the most important things for homeowners to adddress before they put their home on the market.

In [48]:
ss = StandardScaler()

In [56]:
ss.fit(model5_x)

model5_x_scaled = ss.transform(model5_x)

In [57]:
scaled_model = sm.OLS(target, model5_x_scaled).fit()

In [59]:
scaled_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.222
Model:                            OLS   Adj. R-squared (uncentered):              0.222
Method:                 Least Squares   F-statistic:                              513.2
Date:                Sun, 10 Jul 2022   Prob (F-statistic):                        0.00
Time:                        16:50:05   Log-Likelihood:                     -3.1712e+05
No. Observations:               21597   AIC:                                  6.343e+05
Df Residuals:                   21585   BIC:                                  6.344e+05
Df Model:                          12                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const               0          0        nan        nan           0           0
x1          8123.5271   5675.052      1.431      0.152   -2999.994    1.92e+04
x2          5.491e+04   4192.159     13.099      0.000    4.67e+04    6.31e+04
x3          3.779e+04   4491.898      8.414      0.000     2.9e+04    4.66e+04
x4          2.195e+04   4277.420      5.131      0.000    1.36e+04    3.03e+04
x5          1.474e+05   6536.028     22.557      0.000    1.35e+05     1.6e+05
x6         -6.396e+04   5356.720    -11.941      0.000   -7.45e+04   -5.35e+04
x7          1.222e+04   4100.891      2.979      0.003    4178.532    2.03e+04
x8          7.726e+04   4108.323     18.805      0.000    6.92e+04    8.53e+04
x9         -1.262e+04   4695.995     -2.687      0.007   -2.18e+04   -3414.115
x10         4.244e+04   6198.520      6.846      0.000    3.03e+04    5.46e+04
x11        -7764.2800   5713.300     -1.359      0.174    -1.9e+04    3434.211
x12         1.268e+05   5257.269     24.112      0.000    1.16e+05    1.37e+05
==============================================================================
Omnibus:                    13964.327   Durbin-Watson:                   0.241
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1001147.954
Skew:                           2.361   Prob(JB):                         0.00
Kurtosis:                      36.019   Cond. No.                          inf
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

So grade, livable area, latitude, waterfront, sqft_living 15, view, condition, yr_renovated, and sqqft_lot all are positive drivers of price, in order to most to least effect.

Grade, livable area, condition, and yr_renovated are the ones that homeowners have control over, in order of most to least impactful.